## BWAS_Python
In this notebook, I go through an example of how to do Brain-wide association studies (BWAS) in Python. In particular, to compare c-fos cell counts across multiple brain regions across treatment groups. I use two methods: linear regression and negative binomial regression.

This is an attempt to reproduce the results by Will Townes performed in R for Jess Verpeut's 2021 Cell Reports paper. See his (potentially still private) repo: https://github.com/willtownes/neuro (contact at ftownes@princeton.edu).

**NB:** For linear regression, the results are nearly identical between Python and R. For negative binomial regression, the standard error estimates are quite different between Python and R. The estimates themselves are relatively consistent, however. A detailed comparison between these two methods has not been done, nor is it well understood.

- Author: Austin Hoag (with massive help from Will Townes)
- Date: August 25, 2021

In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from patsy import dmatrix

## Example using whole-brain c-fos cell counts

In [2]:
data_file = '../data/Jess_cfos_total_and_fractional_counts-total_122_regions.csv'

In [3]:
df = pd.read_csv(data_file)
df

,brain,batch,condition,Anterior amygdalar area,Basolateral amygdalar nucleus,Central amygdalar nucleus,Cortical amygdalar area,Intercalated amygdalar nucleus,Lateral amygdalar nucleus,Posterior amygdalar nucleus,...,Supragenual nucleus,Supratrigeminal nucleus,Tegmental reticular nucleus,Motor nucleus of trigeminal,Laterodorsal tegmental nucleus,Nucleus incertus,Pontine reticular nucleus,Nucleus raphe pontis,Subceruleus nucleus,Sublaterodorsal nucleus
0,an011,202010_cfos,acquisition_day1,156,2173,2163,736,160,601,293,...,0,263,223,137,221,95,619,31,54,40
1,an012,202010_cfos,acquisition_day1,350,2327,1595,781,224,816,292,...,0,98,165,225,166,36,870,45,14,31
2,an013,202010_cfos,acquisition_day1,569,2517,1224,824,153,704,521,...,0,235,365,183,221,85,834,111,93,64
3,an014,202010_cfos,acquisition_day1,688,2051,1224,523,169,1076,514,...,0,149,224,143,157,106,864,120,17,50
4,an015,202010_cfos,acquisition_day1,150,1452,608,330,100,417,304,...,0,144,272,119,149,51,660,56,25,208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,dadult_pc_crus1_1,201810_adultacutePC_ymaze_cfos,vector_control_reversal,131,311,264,300,18,167,109,...,0,0,0,0,15,0,142,0,0,0
165,dadult_pc_crus1_3,201810_adultacutePC_ymaze_cfos,vector_control_reversal,123,512,298,494,19,113,243,...,0,0,0,0,0,0,110,0,0,0
166,dadult_pc_crus1_4,201810_adultacutePC_ymaze_cfos,vector_control_reversal,99,279,142,143,4,91,40,...,0,0,0,0,0,0,3,0,0,0
167,dadult_pc_lob6_14,201810_adultacutePC_ymaze_cfos,vector_control_reversal,69,711,281,363,28,175,104,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Split predictor variables (batch, condition) from count information
all_predictors = df[['batch','condition']]
all_counts = df.iloc[:,3:]

## Example: CNO_control_reversal (control) vs. CNOnCrusILT (treatment)

The null hypothesis is that c-fos counts are drawn from the same distribution in both of these groups, across all brain regions. We can test this on individual brain regions and also across all brain regions. If doing the latter, we must control for false discovery rate, which we can do with the Benjamini-Hochberg method (see below).  

In [ ]:
# Find all entries from either conditon
condition1 = 'CNO_control_reversal'
condition2 = 'CNOnCrusILT'
condition_mask = (all_predictors['condition']==condition1) | (all_predictors['condition']==condition2)
predictors = all_predictors.loc[condition_mask,:]
counts = all_counts.loc[condition_mask,:]

In [ ]:
counts.shape

In [ ]:
predictors.shape

In [ ]:
predictors

Can see that we have 3 batches here and 2 conditions

In [ ]:
# Use Paty's neat dmatrix() function to create the "design matrix"
# This converts the batch and condition values which are categorical in nature\
# into numeric values
# 
# We only need to do this once per batch/condition combo 
design_matrix = dmatrix("batch + condition", predictors,return_type='dataframe')

In [ ]:
design_matrix

Notice how there are only two batch columns but we started with three batches. That is because If both batch columns here are 0 then the membership is to the third batch. Also note the intercept column which was automatically added. 

In [ ]:
# Do the test for an example brain region
brain_regions = subdf.columns[3:]
brain_region = brain_regions[0]

In [ ]:
sns.displot(pd.concat([predictors,counts],axis=1), x=brain_region, hue="condition",binwidth=100)

## Linear regression

In [ ]:
# Figure out the fractional counts in this brain region
rowsums=np.sum(counts,axis=1)
counts_thisregion = counts[brain_region]
pcounts = counts_thisregion/rowsums

In [ ]:
# Make the Linear Regression model and fit it
mod = sm.OLS(pcounts,design_matrix)
res_LR = mod.fit()

In [ ]:
# Print out a summary of the fit
res_LR.summary()

In [ ]:
# Extract the coefficient from the treatment regressor (the last one in the list)
res_LR.params[-1]

In [ ]:
# Extract the standard error from the treament regressor
res_LR.bse[-1]

In [ ]:
# Extract p-value from the treatment regressor
res_LR.pvalues[-1]

## Compare results to using full Patsy formula

In [ ]:
# Now apply it to our case
df_thisregion = pd.concat([pcounts,predictors],axis=1).rename(
    columns={0:'AAA'})
mod_fullformula = smf.ols(formula='AAA ~ batch + condition', data=df_thisregion)

In [ ]:
fit_fullformula = mod_fullformula.fit()

In [ ]:
fit_fullformula.summary()

This method gives the same results as the method where we precompute the design matrix. Because we will be running this regression for many brain regions and each region has the same predictors, we can use the same design matrix for all regressions. Therefore, it will be faster to use the first method so we will stick with that.

## Negative binomial regression
Here we use total counts instead of fractional counts but must calculate an offset which is the log of the row sums to include in the regression. Otherwise the inputs to the regression are the same. We can use that same design matrix.

Here is a good explanation to help interpret the outputs of the model (see below): https://stats.idre.ucla.edu/sas/output/negative-binomial-regression/

In [ ]:
offsets = np.log(rowsums)

In [ ]:
# Make the negative binomial model and fit it
nb_mod = sm.GLM(counts_thisregion, design_matrix,family=sm.families.NegativeBinomial(),offset=offsets)
nb_fit = nb_mod.fit()

In [ ]:
# Print out a summary of the fit, you can extract the fit parameters in the exact same way
# as we did for the linear model
nb_fit.summary()

In [ ]:
# Make sure we get the same result as the full patsy formula
# Now apply it to our case
df_thisregion = pd.concat([counts_thisregion,predictors],axis=1).rename(
    columns={'Anterior amygdalar area':'AAA'})
nb_mod_fullformula = smf.glm(formula='AAA ~ batch + condition', data=df_thisregion,
                         family=sm.families.NegativeBinomial(),offset=offsets)

In [ ]:
nb_fit_fullformula = nb_mod_fullformula.fit()

In [ ]:
nb_fit_fullformula.summary()

Again, we get the exact same results.

# Arbitrary conditions, looping over all brain regions
Here we will write a function that can accept any two conditions and will run regressions over all brain regions for all animals in all batches of those two conditions.

We will calculate adjusted p-values using the Benjamini-Hochberg method since we will be running N tests, where N is the number of brain regions.

In [15]:
def bh_correction(p_values):
    """ 
    ---DESCRIPTION---
    Benjamini-Hochberg correction of p-values
    This is not well tested. 
    ---INPUT---
    p_values:           a list of p-values, can be unsorted but must not have nans
    ---OUTPUT---
    adjusted_p_values:  a list of adjusted p-values sorted from lowest to highest
    """
    # sort the p-values
    sorted_p_values = sorted(p_values)
    # Make an empty array to fill in
    adjusted_p_values = np.zeros_like(sorted_p_values)
    # Fill them from highest to lowest
    # First adjusted p-value is just highest p-value
    adjusted_p_values[-1] = sorted_p_values[-1]
    for p_value_index in range(len(sorted_p_values)-2,-1,-1):
        next_highest = adjusted_p_values[p_value_index+1]
        rank_current = p_value_index+1
        mod_current = sorted_p_values[p_value_index] * (len(sorted_p_values)/rank_current)
        adjusted_p_value = min(next_highest,mod_current)
        adjusted_p_values[p_value_index] = adjusted_p_value
    return adjusted_p_values

In [16]:
def linear_regression(df,condition1,condition2):
    """ 
    ---DESCRIPTION---
    Perform linear regression between condition1 (assumed to be control) and
    condition2 (assumed to be treatment). Uses all batches found in both conditions.
    This is not well tested.
    ---INPUT---
    df:           an NxK dataframe where N rows are the animals and K are the columns, 
                  which must contain the batch, condition and counts in each brain region
    condition1:   a string describing one of the conditions in the dataframe (the control)
    condition2:   a string describing one of the conditions in the dataframe (the treatment)
    ---OUTPUT--
    lr_df:        A dataframe containing the results from the regression on all brain regions in df
    Also Saves to a CSV file called: '../data/{condition1}-{condition2}-pcounts-linreg.csv'
    """
    all_predictors = df[['batch','condition']]
    all_counts = df.iloc[:,3:]
    brain_regions = df.columns[3:]
    
    condition_mask = (all_predictors['condition']==condition1) | (all_predictors['condition']==condition2)
    predictors = all_predictors.loc[condition_mask,:]
    design_matrix = dmatrix("batch + condition", predictors,return_type='dataframe')
    counts = all_counts.loc[condition_mask,:]
    rowsums=np.sum(counts,axis=1)
    result_list = []
    for ii in range(len(brain_regions)):
        brain_region=brain_regions[ii]
        result_dict = {
            'region_idx':ii,
            'control':condition1,
            'treatment':condition2,
            'region':brain_region
        }
        counts_thisregion = counts[brain_region]
        pcounts = counts_thisregion/rowsums
        res_LR = sm.OLS(pcounts,design_matrix).fit()
        estimate = res_LR.params[-1]
        stderr = res_LR.bse[-1]
        pvalue = res_LR.pvalues[-1]
        zscore = estimate/stderr
        if np.isnan(zscore) or np.isnan(pvalue):
            status="failed"
        else:
            status="success"
        result_dict['Estimate'] = estimate
        result_dict['Std. Error'] = stderr
        result_dict['t value'] = zscore
        result_dict['Pr(>|z|)'] = pvalue
        result_dict['status'] = status
        result_list.append(result_dict)
    # Now calculate adjusted pvalues
    # First sort the whole result_list by p-value, keeping in mind that there can be nans
    sorted_result_list = sorted(result_list,
            key=lambda x: float('-inf') if np.isnan(x.get('Pr(>|z|)')) else x.get('Pr(>|z|)')) 
    sorted_p_values = np.array([d.get('Pr(>|z|)') for d in sorted_result_list])
    # separate out nans (which are at the beginning of the list due to how we sorted)
    p_values_clean = sorted_p_values[~np.isnan(sorted_p_values)]
    p_values_nan = sorted_p_values[np.isnan(sorted_p_values)]
    # calculate the fdr adjusted p-values
    adjusted_p_values_no_nans = bh_correction(p_values_clean)
    # add back the nans to the beginning of the list
    adj_p_values = np.concatenate([p_values_nan,adjusted_p_values_no_nans])
    # add the fdr adjusted p-value into the list of our results
    for ii in range(len(sorted_result_list)):
        sorted_result_list[ii]['fdr_adj_pval'] = adj_p_values[ii]
    # Finally, sort back to original order, the one using brain region index as key
    final_result_list = sorted(sorted_result_list,key=lambda x: x.get('region_idx'))
    # Make pandas dataframe to make it easier to save to file
    lr_df = pd.DataFrame(final_result_list)
    lr_df.set_index('region_idx',inplace=True)
    # reorder columns
    neworder = ['control','treatment','region','Estimate','Std. Error','t value','Pr(>|z|)','fdr_adj_pval','status']
    lr_df=lr_df.reindex(columns=neworder)
    # Save to csv
    savename = f'../data/{condition1}-{condition2}-pcounts-linreg.csv'
    lr_df.to_csv(savename,index=False)
    print(f"Saved {savename}")
    return lr_df

In [17]:
lr_df = linear_regression(df=df,condition1 = 'CNO_control_reversal',condition2 = 'CNOnCrusILT')

Saved ../data/CNO_control_reversal-CNOnCrusILT-pcounts-linreg.csv


<ipython-input-16-0d4154d7e782>:40: RuntimeWarning: invalid value encountered in double_scalars
  zscore = estimate/stderr
<ipython-input-16-0d4154d7e782>:40: RuntimeWarning: invalid value encountered in double_scalars
  zscore = estimate/stderr
<ipython-input-16-0d4154d7e782>:40: RuntimeWarning: invalid value encountered in double_scalars
  zscore = estimate/stderr
<ipython-input-16-0d4154d7e782>:40: RuntimeWarning: invalid value encountered in double_scalars
  zscore = estimate/stderr
<ipython-input-16-0d4154d7e782>:40: RuntimeWarning: invalid value encountered in double_scalars
  zscore = estimate/stderr
<ipython-input-16-0d4154d7e782>:40: RuntimeWarning: invalid value encountered in double_scalars
  zscore = estimate/stderr
<ipython-input-16-0d4154d7e782>:40: RuntimeWarning: invalid value encountered in double_scalars
  zscore = estimate/stderr
<ipython-input-16-0d4154d7e782>:40: RuntimeWarning: invalid value encountered in double_scalars
  zscore = estimate/stderr
<ipython-input-1

In [18]:
def nb_regression(df,condition1,condition2):
    """ 
    ---DESCRIPTION---
    Perform negative binomial regression between condition1 (assumed to be control) and
    condition2 (assumed to be treatment). Uses all batches found in both conditions.
    This is not well tested.
    ---INPUT---
    df:           an NxK dataframe where N rows are the animals and K are the columns, 
                  which must contain the batch, condition and counts in each brain region
    condition1:   a string describing one of the conditions in the dataframe (the control)
    condition2:   a string describing one of the conditions in the dataframe (the treatment)
    ---OUTPUT--
    nb_df:        A dataframe containing the results from the regression on all brain regions in df

    Saves to a CSV file called: '../data/{condition1}-{condition2}-counts-nbreg.csv'
    """
    all_predictors = df[['batch','condition']]
    all_counts = df.iloc[:,3:]
    brain_regions = df.columns[3:]
    
    condition_mask = (all_predictors['condition']==condition1) | (all_predictors['condition']==condition2)
    predictors = all_predictors.loc[condition_mask,:]
    design_matrix = dmatrix("batch + condition", predictors,return_type='dataframe')
    counts = all_counts.loc[condition_mask,:]
    rowsums=np.sum(counts,axis=1)
    offsets = np.log(rowsums)
    result_list = []
    for ii in range(len(brain_regions)):
        brain_region=brain_regions[ii]
        result_dict = {
            'region_idx':ii,
            'control':condition1,
            'treatment':condition2,
            'region':brain_region
        }
        counts_thisregion = counts[brain_region]
        try:
            res_nb = sm.GLM(counts_thisregion, design_matrix,family=sm.families.NegativeBinomial(),offset=offsets).fit()
            estimate = res_nb.params[-1]
            stderr = res_nb.bse[-1]
            pvalue = res_nb.pvalues[-1]
            zscore = estimate/stderr
            status="success"
        except:
            estimate = np.nan
            stderr = np.nan
            pvalue = np.nan
            zscore = np.nan
            status="failed"
        result_dict['Estimate'] = estimate
        result_dict['Std. Error'] = stderr
        result_dict['z value'] = zscore
        result_dict['Pr(>|z|)'] = pvalue
        result_dict['status'] = status
        result_list.append(result_dict)
#     print(result_list)
    # Now calculate adjusted pvalues
    # First sort the whole result_list by p-value, keeping in mind that there can be nans
    sorted_result_list = sorted(result_list,
            key=lambda x: float('-inf') if np.isnan(x.get('Pr(>|z|)')) else x.get('Pr(>|z|)')) 
    sorted_p_values = np.array([d.get('Pr(>|z|)') for d in sorted_result_list])
    # separate out nans (which are at the beginning of the list due to how we sorted)
    p_values_clean = sorted_p_values[~np.isnan(sorted_p_values)]
    p_values_nan = sorted_p_values[np.isnan(sorted_p_values)]
    # calculate the fdr adjusted p-values
    adjusted_p_values_no_nans = bh_correction(p_values_clean)
    # add back the nans to the beginning of the list
    adj_p_values = np.concatenate([p_values_nan,adjusted_p_values_no_nans])
    # add the fdr adjusted p-value into the list of our results
    for ii in range(len(sorted_result_list)):
        sorted_result_list[ii]['fdr_adj_pval'] = adj_p_values[ii]
    # Finally, sort back to original order, the one using brain region index as key
    final_result_list = sorted(sorted_result_list,key=lambda x: x.get('region_idx'))
    # Make pandas dataframe to make it easier to save to file
    nb_df = pd.DataFrame(final_result_list)
    nb_df.set_index('region_idx',inplace=True)
    # reorder columns
    neworder = ['control','treatment','region','Estimate','Std. Error','z value','Pr(>|z|)','fdr_adj_pval','status']
    nb_df=nb_df.reindex(columns=neworder)
    # Save to csv
    savename = f'../data/{condition1}-{condition2}-counts-nbreg.csv'
    nb_df.to_csv(savename,index=False)
    print(f"Saved {savename}")
    return nb_df

In [19]:
nb_df = nb_regression(df=df,condition1 = 'CNO_control_reversal',condition2 = 'CNOnCrusILT')

/home/ahoag/anaconda3/envs/cfos/lib/python3.8/site-packages/statsmodels/genmod/families/family.py:1227: RuntimeWarning: invalid value encountered in true_divide
  endog_mu = self._clean(endog / mu)
/home/ahoag/anaconda3/envs/cfos/lib/python3.8/site-packages/statsmodels/genmod/families/family.py:1227: RuntimeWarning: invalid value encountered in true_divide
  endog_mu = self._clean(endog / mu)
/home/ahoag/anaconda3/envs/cfos/lib/python3.8/site-packages/statsmodels/genmod/families/family.py:1227: RuntimeWarning: invalid value encountered in true_divide
  endog_mu = self._clean(endog / mu)
/home/ahoag/anaconda3/envs/cfos/lib/python3.8/site-packages/statsmodels/genmod/families/family.py:1227: RuntimeWarning: invalid value encountered in true_divide
  endog_mu = self._clean(endog / mu)
/home/ahoag/anaconda3/envs/cfos/lib/python3.8/site-packages/statsmodels/genmod/families/family.py:1227: RuntimeWarning: invalid value encountered in true_divide
  endog_mu = self._clean(endog / mu)
/home/ahoa

Saved ../data/CNO_control_reversal-CNOnCrusILT-counts-nbreg.csv


/home/ahoag/anaconda3/envs/cfos/lib/python3.8/site-packages/statsmodels/genmod/families/family.py:1227: RuntimeWarning: invalid value encountered in true_divide
  endog_mu = self._clean(endog / mu)
